# 192. Word Frequency - Bash解法

## 問題概要

テキストファイル `words.txt` から各単語の出現頻度を集計し、頻度の降順で出力する問題です。

---

## 解答（スクリプト版）

`wordfreq.sh`（Bash, POSIX ツールのみ）

```bash
#!/usr/bin/env bash
set -euo pipefail

# 使い方: ./wordfreq.sh [path/to/words.txt]
# 引数が未指定なら ./words.txt を読む
input="${1:-words.txt}"

# 1) 全ての空白(スペース/タブ/改行など)を改行にし、連続空白は1つに圧縮
# 2) ソート
# 3) uniq -c で頻度集計
# 4) 頻度(第1列)で数値降順ソート
# 5) "単語 頻度" の並びに整形
LC_ALL=C tr -s '[:space:]' '\n' < "$input" \
  | sort \
  | uniq -c \
  | sort -nr \
  | awk '{print $2, $1}'
```

### 実行方法

```bash
chmod +x wordfreq.sh
./wordfreq.sh                 # カレントの words.txt を集計
# もしくは
./wordfreq.sh /path/to/words.txt
```

---

## 解答（パイプのみの1行版）

```bash
LC_ALL=C tr -s '[:space:]' '\n' < words.txt | sort | uniq -c | sort -nr | awk '{print $2, $1}'
```

---

## 入出力例

### 入力 (`words.txt`)

```text
the day is sunny the the
the sunny is is
```

### 出力

```text
the 4
is 3
sunny 2
day 1
```

---

## 処理フロー図解

```mermaid
flowchart LR
  A["words.txt<br/>入力ファイル"] --> B["<code>tr -s &#91;:space:&#93; \\n</code><br/>全ての空白→改行<br/>連続空白を1つに圧縮"]
  B --> C["<code>sort</code><br/>辞書順整列"]
  C --> D["<code>uniq -c</code><br/>連続同一語をカウント"]
  D --> E["<code>sort -nr</code><br/>頻度で降順ソート"]
  E --> F["<code>awk &#123;print $2, $1&#125;</code><br/>「単語 頻度」形式に整形"]
  F --> G["結果出力"]
```

---

## ステップ別の処理詳細

### 入力データ

```text
the day is sunny the the
the sunny is is
```

### ステップ1: `tr -s '[:space:]' '\n'`

全ての空白文字（スペース・タブ・改行）を改行に変換し、連続する空白は1つに圧縮します。

```text
the
day
is
sunny
the
the
the
sunny
is
is
```

### ステップ2: `sort`

単語を辞書順に整列します（`uniq -c` は連続した同一行のみカウントするため必須）。

```text
day
is
is
is
sunny
sunny
the
the
the
the
```

### ステップ3: `uniq -c`

連続する同一単語をカウントします。

```text
      1 day
      3 is
      2 sunny
      4 the
```

### ステップ4: `sort -nr`

頻度（第1列）で数値降順ソートします。

```text
      4 the
      3 is
      2 sunny
      1 day
```

### ステップ5: `awk '{print $2, $1}'`

「単語 頻度」の形式に整形します。

```text
the 4
is 3
sunny 2
day 1
```

---

## アルゴリズムの解説

### なぜこの順番なのか？

1. **`tr` で正規化**
   - 様々な空白文字（スペース・タブ・改行）を統一的に処理
   - 連続空白の圧縮により空行を防止

2. **最初の `sort` が必須**
   - `uniq -c` は**連続した**同一行のみカウント
   - 事前に整列することで同じ単語を隣接させる

3. **`uniq -c` で集計**
   - 連続する同一単語の出現回数をカウント
   - 出力形式: `<頻度> <単語>`

4. **`sort -nr` で降順**
   - `-n`: 数値としてソート
   - `-r`: 降順（reverse）

5. **`awk` で整形**
   - 列の順序を入れ替え: `$2 $1` → `<単語> <頻度>`

---

## 代替解法（awk メイン）

`awk` の連想配列を使った方法：

```bash
awk '{for(i=1;i<=NF;i++) c[$i]++} END{for(w in c) print w, c[w]}' words.txt \
  | LC_ALL=C sort -k2,2nr
```

### 処理の流れ

1. `awk` で各単語をカウント
   - `NF`: 行内のフィールド数（空白区切り）
   - `c[$i]++`: 連想配列でカウント

2. `END` ブロックで出力
   - `for(w in c)`: 全ての単語をループ
   - `print w, c[w]`: 単語と頻度を出力

3. `sort -k2,2nr` で頻度降順ソート
   - `-k2,2`: 第2列（頻度）でソート
   - `n`: 数値ソート
   - `r`: 降順

---

## パフォーマンス最適化のポイント

### 1. ロケール設定

```bash
LC_ALL=C
```

- C ロケールを使用することで `sort` が高速化
- バイト単位の比較により安定した動作

### 2. 空行の除去（必要に応じて）

`tr -s` を使っていれば基本的に不要ですが、念のため：

```bash
... | grep -v '^$' | ...
```

### 3. 入力ファイルの柔軟な指定

スクリプト版では引数でファイルパスを指定可能：

```bash
input="${1:-words.txt}"
```

---

## 応用例

### 圧縮ファイルの処理

```bash
zcat compressed.txt.gz | tr -s '[:space:]' '\n' | sort | uniq -c | sort -nr | awk '{print $2, $1}'
```

### ストリーム処理

```bash
curl -s https://example.com/text.txt | tr -s '[:space:]' '\n' | sort | uniq -c | sort -nr | awk '{print $2, $1}'
```

### 大文字小文字を区別しない

```bash
LC_ALL=C tr -s '[:space:]' '\n' < words.txt \
  | tr '[:upper:]' '[:lower:]' \
  | sort \
  | uniq -c \
  | sort -nr \
  | awk '{print $2, $1}'
```

---

## よくある質問

### Q1: `tr -s` の `-s` オプションは何をする？

**A:** `-s` (squeeze) は連続する文字を1つに圧縮します。

```bash
# 例: 連続するスペースを1つに
echo "a  b   c" | tr -s ' '
# 出力: a b c
```

### Q2: なぜ `LC_ALL=C` を使うのか？

**A:** 
- ロケール依存の文字比較を避ける
- バイト単位の比較で高速化
- 環境による動作の違いを防ぐ

### Q3: `uniq -c` の出力形式は？

**A:** `<頻度><スペース><単語>` の形式で出力されます。

```text
      4 the
      3 is
```

先頭にスペースが入るため、`awk` で列を入れ替える際は `$1` が頻度、`$2` が単語になります。

---

## Mermaid図の注意点

Mermaid でコマンドを含むラベルを書く際の安全な記法：

### 特殊文字のエスケープ

- 角かっこ `[` `]` → `&#91;` `&#93;`
- 波かっこ `{` `}` → `&#123;` `&#125;`
- バックスラッシュ `\` → `\\`
- シングルクォート `'` → `&apos;`（必要な場合）

### 推奨記法

```mermaid
flowchart LR
  A["ノード名"] --> B["<code>コマンド</code><br/>説明文"]
```

- ラベル全体を二重引用符 `["..."]` で囲む
- コマンド部分は `<code>` タグで囲む
- 改行は `<br/>` を使用

---

## まとめ

この問題の解法ポイント：

1. **`tr`** で空白を正規化
2. **`sort`** で同一単語を隣接させる
3. **`uniq -c`** で頻度をカウント
4. **`sort -nr`** で頻度降順ソート
5. **`awk`** で出力形式を整形

シンプルな POSIX ツールの組み合わせで効率的に処理できます。

主な改善点：
1. 重複セクションを完全に削除
2. 構造を論理的に整理（問題→解答→詳細→応用）
3. Mermaid図を1つに統一（安全な記法を使用）
4. よくある質問セクションを追加
5. 応用例を充実
6. Mermaid記法の注意点を最後にまとめ